# Utility Chains Overview

## Summarizing Documents

### load_summarize_chain

In [11]:
import os
import pprint as pp
from pathlib import Path
from dotenv import find_dotenv, load_dotenv

load_dotenv(Path('../../.env'))

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain


In [5]:
llm = OpenAI(temperature=0.9)

In [6]:
# Reading the document
with open("sample.txt") as f:
    data = f.read()

In [ ]:
# Split text - When it comes to document processing,
# breaking a large document into smaller, more manageable 
# chunks is essential
text_splitter = CharacterTextSplitter()
texts = text_splitter.split_text(data)

In [8]:
# Create multiple documents
docs = [Document(page_content=t) for t in texts]

In [9]:
docs

[Document(page_content="Title: The Computer: Revolutionizing the World of Technology\n\nIntroduction:\nThe computer is a marvel of human ingenuity that has revolutionized the world in countless ways. From its humble beginnings as a complex calculating machine to its current status as a ubiquitous tool in every aspect of our lives, the computer has transformed how we work, communicate, learn, and entertain ourselves. This essay explores the evolution, impact, and future potential of computers in shaping our modern world.\n\nThe Birth of the Computer:\nThe computer has its roots in the early 19th century when inventors and mathematicians began conceptualizing machines capable of automating complex calculations. However, it was not until the mid-20th century that the first electronic digital computers were developed. Pioneers such as Alan Turing, John von Neumann, and Grace Hopper made significant contributions to the field, laying the groundwork for the computers we know today.\n\nThe Ev

To create an instance of load_summarizer_chain, we need to provide three arguments. <br><br>Firstly, we need to pass the desired large language model that will be used to query the user input. Secondly, we specify the type of langchain chain to be used for summarizing documents.<br> Lastly, we can set the verbose argument to True if we want to see all the intermediate steps involved in processing the user request and generating the output.<font>

In [10]:
# Reduce the size of the documents.
# MapReduce: Summary of each, then make a summary of the both
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"Title: The Computer: Revolutionizing the World of Technology

Introduction:
The computer is a marvel of human ingenuity that has revolutionized the world in countless ways. From its humble beginnings as a complex calculating machine to its current status as a ubiquitous tool in every aspect of our lives, the computer has transformed how we work, communicate, learn, and entertain ourselves. This essay explores the evolution, impact, and future potential of computers in shaping our modern world.

The Birth of the Computer:
The computer has its roots in the early 19th century when inventors and mathematicians began conceptualizing machines capable of automating complex calculations. However, it was not until the mid-20th century that the first electronic digital computers were developed. Pioneers such as Alan Turing, John von Neumann, a

' This essay traces the evolution of the computer from its inception in the early 19th century to its current form, and its impact on modern life. Computers have revolutionized work, communication, education, and entertainment, and have the potential to continue doing so in the future. Human innovation has made the computer a compact and powerful tool with immense potential, and with responsible development and usage, it can positively shape our lives for generations to come.'

## HTTP Requests

### LLMRequestsChain

In [12]:
template = """
Extract the answer to the question '{query}' or say "not found" if the information is not available.
{requests_result}
"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)

In [13]:
llm=OpenAI()
chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=PROMPT))

Preparing the question & inputs to the http request<font>

In [15]:
question = "What is the capital of india?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}

In [16]:
chain(inputs)

{'query': 'What is the capital of india?',
 'url': 'https://www.google.com/search?q=What+is+the+capital+of+india?',
 'output': '\nNew Delhi'}

In [17]:
# Using beautifulsoup to scrape data from
# the google searchs.
import inspect
print(inspect.getsource(chain._call))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        from bs4 import BeautifulSoup

        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        # Other keys are assumed to be needed for LLM prediction
        other_keys = {k: v for k, v in inputs.items() if k != self.input_key}
        url = inputs[self.input_key]
        res = self.requests_wrapper.get(url)
        # extract the text from the html
        soup = BeautifulSoup(res, "html.parser")
        other_keys[self.requests_key] = soup.get_text()[: self.text_length]
        result = self.llm_chain.predict(
            callbacks=_run_manager.get_child(), **other_keys
        )
        return {self.output_key: result}



In [18]:
? chain

Signature:     
 chain(
    inputs: Union[Dict[str, Any], Any],
    return_only_outputs: bool = False,
    callbacks: Union[List[langchain.callbacks.base.BaseCallbackHandler], langchain.callbacks.base.BaseCallbackManager, NoneType] = None,
    *,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    run_name: Optional[str] = None,
    include_run_info: bool = False,
) -> Dict[str, Any]
Type:           LLMRequestsChain
String form:    memory=None callbacks=None callback_manager=None verbose=False tags=None metadata=None llm_chain= <...> e, auth=None) text_length=8000 requests_key='requests_result' input_key='url' output_key='output'
File:           ~/anaconda3/envs/masterclass/lib/python3.9/site-packages/langchain/chains/llm_requests.py
Docstring:      Chain that requests a URL and then uses an LLM to parse results.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data 